# [IA Frameworks](https://github.com/wikistat/AI-Frameworks) - Introduction to Deep Reinforcement Learning 

<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" width=400, style="max-width: 150px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" width=400,  style="float:right;  display: inline" alt="IMT"/> </a>
    
</center>

# Part 1a : Q-Learning
The objectives of this noteboks are the following : 

* A sa reminder implement Q-iteration and Q-Learning on simple Markov Decision Process

# Import librairies

In [1]:
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym

# Markov Decision Process

## Definition

We will first define a simple markov process on wich we will apply Q-learning algorithm.

here is an illustration of the MDP that we will define.

![images](images/mdp.png)

### Transition probabilities

We first define the different **transition probabilities** for each $(s,a,s')$ combination where
* $s$ is the `from_state`
* $a$ is the `action` taken
* $s$ is the `to_state`

We store the **transition probabilities** within a python list and use pandas to visualize it better

In [2]:
transition_probabilities = [
        [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]], 
        [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
        [None, [0.8, 0.1, 0.1], None],
    ]

transition_probabilities_df = pd.DataFrame(transition_probabilities).rename_axis('Actions', axis=1)
transition_probabilities_df.index.name="State"
transition_probabilities_df

Actions,0,1,2
State,,,
0,"[0.7, 0.3, 0.0]","[1.0, 0.0, 0.0]","[0.8, 0.2, 0.0]"
1,"[0.0, 1.0, 0.0]",None,"[0.0, 0.0, 1.0]"
2,None,"[0.8, 0.1, 0.1]",None


### Rewards 

We also define the **rewards** for each $(s,a,s')$ combination.

In [3]:
rewards = [
        [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
        [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
        [[0, 0, 0], [+40, 0, 0], [0, 0, 0]],
    ]

rewards_df = pd.DataFrame(rewards).rename_axis('Actions', axis=1)
rewards_df.index.name="State"
rewards_df

Actions,0,1,2
State,,,
0,"[10, 0, 0]","[0, 0, 0]","[0, 0, 0]"
1,"[0, 0, 0]","[0, 0, 0]","[0, 0, -50]"
2,"[0, 0, 0]","[40, 0, 0]","[0, 0, 0]"


### Actions

And the list of possible **actions** that can be taken at each state.

In [4]:
possible_actions = [[0, 1, 2], [0, 2], [1]]

possible_actions_df = pd.DataFrame([[x] for x in possible_actions], columns=["List of possible actions"])
possible_actions_df.index.name="State"
possible_actions_df

,List of possible actions
State,
0,"[0, 1, 2]"
1,"[0, 2]"
2,[1]


## Class environment

Finally we define now a class that will act as a Gym environment. 

* The environement is the MDP.
* The observation is the current step.
* The action possible are the three actions we previously define.

In [5]:
class MDPEnvironment(object):
    def __init__(self, start_state=0):
        self.start_state=start_state
        self.reset()
    def reset(self):
        self.total_rewards = 0
        self.state = self.start_state
    def step(self, action):
        next_state = np.random.choice(range(3), p=transition_probabilities[self.state][action])
        reward = rewards[self.state][action][next_state]
        self.state = next_state
        self.total_rewards += reward
        return self.state, reward

## Hard Coded Policy

Let's first implement a random policy, as a baseline we want to improve.

We play this policy one thousand of time.

In [6]:
def policy_random(state):
    return np.random.choice(possible_actions[state])


def run_episode(policy, n_steps, start_state=0):
    env = MDPEnvironment()
    for step in range(n_steps):
        action = policy(env.state)
        state, reward = env.step(action)
    return env.total_rewards


all_score = []
for episode in range(1000):
    all_score.append(run_episode(policy_random, n_steps=100))
print("Summary: mean={:.1f}, std={:1f}, min={}, max={}".format(np.mean(all_score), np.std(all_score), np.min(all_score), np.max(all_score)))


Summary: mean=-29.2, std=88.485931, min=-370, max=220


**Exercise** Which policy would be the safest? The more risky? Implement it and test it. What can you say about their results?

In [13]:
# %load solutions/hard_coded_policy.py

Summary: mean=119.8, std=137.600028, min=-450, max=550
Summary: mean=24.0, std=27.711341, min=0, max=210


## Q-value iteration algorithm

Let's know try to find the best policy! <br>
Because we know all the **transition probabilities** and **reward values** for each $(s,a,s')$ combination we can compute the this best policy using the **Q-iteration algorithm**

$$Q_{k+1}(s,a) \leftarrow  \sum_{s'}P^a_{s,s'}\big[ R(s,a,s') + \gamma \cdot max_{a'}~Q_k(s',a') \big]$$

Let's first instantiate the Q-values table.

In [11]:
n_states = 3
n_actions = 3
gamma = 0.99  #<-- The discount rate
q_values = np.full((n_states, n_actions), -np.inf) 
for state, action in enumerate(possible_actions):
    q_values[state][action]=0
q_values

array([[  0.,   0.,   0.],
       [  0., -inf,   0.],
       [-inf,   0., -inf]])

**Exercise**: Implement the Q-iteration algorithm to find the optimal Q-values for each action-state couple.

In [16]:
# %load solutions/q_iteration.py

We are now able to retrieved the best policy for each state!

In [32]:
optimal_action_per_state = np.argmax(q_values,axis=1)
optimal_action_per_state

array([0, 2, 1])

**Exercise**: Implement the best policy from this q_values computed

In [33]:
# %load solutions/optimal_policy.py

In [29]:
all_totals = []
for episode in range(1000):
    all_totals.append(run_episode(optimal_policy, n_steps=100))
print("Summary: mean={:.1f}, std={:1f}, min={}, max={}".format(np.mean(all_totals), np.std(all_totals), np.min(all_totals), np.max(all_totals)))
print()

Summary: mean=122.1, std=133.894697, min=-420, max=560



## Q-Learning iteration algorithm

Let's know implement Q-learning algorithm to learn a better policy!

Q-Learning works by watching an agent play (e.g., randomly) and gradually improving its estimates of the Q-Values. 
Once it has accurate Q-Value estimates (or close enough), then the optimal policy consists in choosing the action that has the highest Q-Value (i.e., the greedy policy).

We first initiate:
* the different parameters (learning_rate $\alpha$ and the discount rate $\gamma$}
* The number of step to play
* The exploration policy (random one)
* The Q-values tables

In [47]:
n_states = 3
n_actions = 3
n_steps = 2000
alpha = 0.01  #<-- Learning Rate
gamma = 0.99  #<-- The discount rate


 
exploration_policy = policy_random #<-- Policy that we will play during exploration
q_values = np.full((n_states, n_actions), -np.inf) #<-- Policy that we will be updated
for state, actions in enumerate(possible_actions):
    q_values[state][actions]=0
q_values

array([[  0.,   0.,   0.],
       [  0., -inf,   0.],
       [-inf,   0., -inf]])

**Exercise**
Run *n_steps* over the MDP and update the Q-values table at each step according to the Q-learning iteration algorithm

In [51]:
# %load solutions/q_learning.py

**Question** Run the algorithm over 20000 steps and observe the optimal action per state below. Run it again over 20000 episode. What do you observe?

In [52]:
optimal_action_per_state = np.argmax(q_values,axis=1)
optimal_action_per_state

array([0, 0, 1])

In [53]:
def optimal_policy(state):
    return optimal_action_per_state[state]

Compute its performance.

In [54]:
all_totals = []
for episode in range(1000):
    all_totals.append(run_episode(optimal_policy, n_steps=100))
print("Summary: mean={:.1f}, std={:1f}, min={}, max={}".format(np.mean(all_totals), np.std(all_totals), np.min(all_totals), np.max(all_totals)))
print()

Summary: mean=24.1, std=27.238142, min=0, max=170

